In [49]:
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset



In [48]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token  # Set pad token to eos token
model = GPT2ForSequenceClassification.from_pretrained('gpt2', num_labels=2)

def preprocess_function(examples):
    return tokenizer(
        examples["cleaned_review_text"],
        padding=True,  # Enables dynamic padding
        truncation=True,
        max_length=128  # Ensure consistent input size
    )

tokenizer.pad_token 

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


'<|endoftext|>'

In [50]:
dataset = load_dataset('json', data_files={'train': '/Users/chi/Documents/imdb_spoiler/sampled_preprocessed.json'})
# Split 80% Train, 20% Temp (Validation + Test)
train_test_split = dataset["train"].train_test_split(test_size=0.2, seed=42)

# Further split the 20% Temp into 10% Validation and 10% Test
val_test_split = train_test_split["test"].train_test_split(test_size=0.5, seed=42)

final_dataset = {
    "train": train_test_split["train"],
    "val": val_test_split["train"],  # 10% validation
    "test": val_test_split["test"],  # 10% test
}

tokenized_datasets = {split: data.map(preprocess_function, batched=True) for split, data in final_dataset.items()}


In [51]:
# Training arguments
training_args = TrainingArguments(
    output_dir='saves/results_gpt2',
    num_train_epochs=3,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='saves/logs_gpt2',
)

In [ ]:
tokenized_datasets['train']

datasets.arrow_dataset.Dataset

In [58]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['val'],
    tokenizer=tokenizer,
)

/var/folders/4x/qtt50c_16_j8ntstjbyx20v40000gn/T/ipykernel_10966/1216322619.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [59]:
trainer.train()
results_gpt2 = trainer.evaluate()
print(f"GPT-2 Evaluation Loss: {results_gpt2['eval_loss']:.4f}")

ValueError: The model did not return a loss from the inputs, only the following keys: logits,past_key_values. For reference, the inputs it received are input_ids,attention_mask.